In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
root_dir="/content/drive/MyDrive/BTP/"

In [3]:
import os
import sys
import importlib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.activations import *
import matplotlib.pyplot as plt
from IPython import *
import cv2
from tqdm import *
import pickle
import random

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import *
from drive.MyDrive.BTP.utils import *

In [4]:
var_dtype='float32'

In [5]:
def im(q,ind,ext=".JPG"):
    return "q"+str(q)+"_im"+str(ind)+ext

def op(q,ind,ext=".JPG"):
    return "q"+str(q)+"_op"+str(ind)+ext

def getNames(q):
    return [im(q,i) for i in range(1,5)]+[op(q,i) for i in range(1,5)]

def imshow(img,scale=1):
    plt.imshow((scale*img).astype(np.uint8))
    plt.show()

def imread(fname,target_shape=None):
    img=cv2.imread(fname)
    if target_shape is not None: return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)
    return img

def resize(img,target_shape):
    return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)

In [6]:
def generator(x,y,bsize=32):
    if bsize==-1: yield x,y
    i=0
    n=x.shape[0]
    while i+bsize<=n:
        yield x[i:i+bsize],y[i:i+bsize]
        i+=bsize
    if i<n: yield x[i:],y[i:]

def generator_random(x,y,bsize=32):
    if bsize==-1: yield x,y
    else:
        n=x.shape[0]
        indexes=np.array([i for i in range(n)])
        np.random.shuffle(indexes)
        i=0
        while i+bsize<=n:
            yield x[indexes[i:i+bsize]],y[indexes[i:i+bsize]]
            i+=bsize
        if i<n: yield x[indexes[i:]],y[indexes[i:]]

In [7]:
def convert2vector(imgs,encoder):
    assert len(imgs.shape)==5
    r=[]
    n=imgs.shape[0]
    for i in range(n):
        r.append(encoder(imgs[i]).numpy())
    return np.array(r,dtype=var_dtype)

In [8]:
root_dir="/content/drive/MyDrive/BTP/"
root_data_dir=root_dir+"created_dataset/"
csv=root_dir+"image_cop.csv"

encoder_weights=root_dir+"trained_models/encoder.h5"
rel_decoder_weights=root_dir+"trained_models/rel_decoder.h5"

classifier_weights=root_dir+"trained_models/classifier.h5"
lstm_weights=root_dir+"trained_models/lstm.pkl"

inp_image_size=(64,64)
inp_image_channel=out_image_channel=3
inp_image_shape=inp_image_size+(inp_image_channel,)
batch_size=32
#rel_decoder_input_shape=(6,6,2048)

initilizer=tk.initializers.RandomUniform(minval=-0.1, maxval=0.1, seed=None)

In [9]:
load_encoder_weights=root_dir+"backup/encoder-85.h5"
load_rel_decoder_weights=root_dir+"backup/rel_decoder-latest.h5"

load_classifier_weights=root_dir+"backup/classifier-85.h5"
load_lstm_weights=root_dir+"backup/lstm-85.pkl"

In [ ]:
train_imgs=train=np.load(root_dir+'train_images_arrays.npy').astype(var_dtype)
test_imgs=test=np.load(root_dir+'test_images_arrays.npy').astype(var_dtype)
imshow(train_imgs[0][0]*255)

In [11]:
encoder=getEncoder(trainable=False,weights=load_encoder_weights)

58900480/58889256 [==============================] - 0s 0us/step


In [12]:
recurrent_activation="sigmoid"
insert_time_axis=Lambda(lambda x: tf.expand_dims(x,axis=1))
lstm=LSTM(128,activation=None,recurrent_activation=recurrent_activation,return_sequences=True,return_state=True,input_shape=(1,)+(encoder.output_shape[1:]))#,kernel_initializer=initilizer,bias_initializer='zeros')#recurrent_activation=LeakyReLU(),

In [13]:
x,y,z=lstm(tf.ones(shape=(1,1)+(encoder.output_shape[1:]),dtype=var_dtype))

In [14]:
with open(load_lstm_weights, 'rb') as f:
   lstm.set_weights(pickle.load(f))

In [15]:
classifier_inp=Input(shape=(lstm.units,))
x=Dense(128,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(classifier_inp)
x=Dense(64,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(x)
x=Dense(32,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(x)
#x=Dense(512,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(x)
classifier_out=Dense(1,activation='sigmoid',kernel_initializer=initilizer,bias_initializer='zeros')(x)
classifier=tk.Model(inputs=classifier_inp,outputs=classifier_out)

In [16]:
classifier.load_weights(load_classifier_weights)

In [17]:
def plotAll(images,pred=None,fig_size=(12,8)):
    ind=0
    fig=plt.figure(figsize=fig_size)
    for i in range(1,4+1):
        ax=fig.add_subplot(2,4,i)
        ax.title.set_text("Image "+str(i))
        plt.imshow(images[ind])
        ind+=1
    
    for i in range(5,8+1):
        ax=fig.add_subplot(2,4,i)
        if pred is not None: ax.title.set_text("Opt Score : "+str(pred[i-5]))
        plt.imshow(images[ind])
        ind+=1

    plt.show()

In [ ]:
plotAll(train[0],[0.4,0.3,0.3,0.3])

In [18]:
def show(x_train,print_pred=False,plot_false_only=False):#,apply_recons=True
    for i in range(x_train.shape[0]):
        encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(encoder(x_train[i]),axis=1)

        encoded_vec=tf.concat([encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4],axis=0)

        out,h_t1,c_t1=lstm(insert_time_axis(encoded_img1))
        out,h_t2,c_t2=lstm(insert_time_axis(encoded_img2),initial_state=[h_t1,c_t1])
        out,h_t3,c_t3=lstm(insert_time_axis(encoded_img3),initial_state=[h_t2,c_t2])
        out,h_t4,c_t4=lstm(insert_time_axis(encoded_img4),initial_state=[h_t3,c_t3])


        out_op1,h_t5_op1,c_t5_op1=lstm(insert_time_axis(encoded_op1),initial_state=[h_t4,c_t4])
        out_op2,h_t5_op2,c_t5_op2=lstm(insert_time_axis(encoded_op2),initial_state=[h_t4,c_t4])
        out_op3,h_t5_op3,c_t5_op3=lstm(insert_time_axis(encoded_op3),initial_state=[h_t4,c_t4])
        out_op4,h_t5_op4,c_t5_op4=lstm(insert_time_axis(encoded_op4),initial_state=[h_t4,c_t4])

        classification_lable_op1=classifier(h_t5_op1)
        classification_lable_op2=classifier(h_t5_op2)
        classification_lable_op3=classifier(h_t5_op3)
        classification_lable_op4=classifier(h_t5_op4)

        relation=h_t4



        shape=classification_lable_op1.shape
        dtype=classification_lable_op1.dtype

            
        pred=tf.concat([classification_lable_op1,classification_lable_op2,classification_lable_op3,classification_lable_op4],axis=0)
        if print_pred: print(pred)

        if plot_false_only:
            if pred[0][0]<pred[1][0] or pred[0][0]<pred[2][0] or pred[0][0]<pred[3][0]: plotAll(x_train[i],pred.numpy())
            continue

        plotAll(x_train[i],pred.numpy())

#**Train-Set Examples**

In [ ]:
plot_correct_train_examples=[642,335,238,727,571,497,576]
show(train[plot_correct_train_examples])

In [ ]:
plot_incorrect_train_examples=[0,14,93,124,283,934]
show(train[plot_incorrect_train_examples])

In [ ]:
i=np.random.randint(0,train.shape[0]-1)
for i in range(801,1000):
    print(i)
    show(train[i:i+1],plot_false_only=True)

#**Test-Set Examples**

In [ ]:
plot_correct_test_examples=[385,438,53,45,106,165,202,148,214,102]
show(test[plot_correct_test_examples])

In [ ]:
plot_incorrect_test_examples=[367,97,82,35,31,119,367,359]
show(test[plot_incorrect_test_examples])

In [ ]:
i=np.random.randint(0,test.shape[0]-1)
for i in range(300,400):
    print(i)
    show(test[i:i+1],plot_false_only=True)

In [ ]:
def plotExample(images,fig_size=(12,8)):
    ind=0
    fig=plt.figure(figsize=fig_size)
    for i in range(1,4+1):
        ax=fig.add_subplot(2,4,i)
        ax.title.set_text("Image "+str(i))
        plt.imshow(images[ind])
        ind+=1
    
    for i in range(5,8+1):
        ax=fig.add_subplot(2,4,i)
        ax.title.set_text("Option "+str(i-4))
        plt.imshow(images[ind])
        ind+=1

    plt.show()

In [ ]:
plotExample(train[301])
plotExample(train[231])
plotExample(train[334])
plotExample(train[356])